In [1]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import multiprocessing as mp
from sklearn.model_selection import train_test_split
import torch
from torch import nn
from torch.optim import Adam
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
os.environ["CUDA_VISIBLE_DEVICES"] = '3'

In [2]:
trained_model_path = '/workdir/security/home/junjiehuang2468/paper/trained_models_weight/ember/'
best_trained_model = '2022-01-18 14:55/2w_epoch:0_test_acc:0.890858.pt'
data_path = "/workdir/security/home/junjiehuang2468/paper/data/ember2018/"
train_data_path = data_path + "malwares/" 
test_data_path = data_path + "test_malwares/" 

In [3]:
CUDA = True if torch.cuda.is_available() else False
NUM_WORKERS = 24
BATCH_SIZE = 80
LEAVE_BIT_NUMBER = 20000
KERNEL_SIZE = 500

In [4]:
LEAVE_BIT_NUMBER_DICT = {10000:'1w',20000:'2w'}

In [5]:
trainset = pd.read_csv(data_path + 'train_dataset.csv')
validset = pd.read_csv(data_path + 'valid_dataset.csv')
testset = pd.read_csv(data_path + 'test_dataset.csv')
testset = testset.iloc[np.argwhere(testset['labels'].values == 1).squeeze(),:]

In [6]:
class ExeDataset(Dataset):
    def __init__(self, malware_names, data_path, labels, leave_bit_num):
        self.malware_names = malware_names
        self.data_path = data_path
        self.labels = labels
        self.leave_bit_num = leave_bit_num

    def __len__(self):
        return len(self.malware_names)

    def __getitem__(self, idx):
        with open(self.data_path + self.malware_names[idx] + '.txt','rb') as fp:
            data = [bit+1 for bit in fp.read()[:self.leave_bit_num]]
            padding = [0]*(self.leave_bit_num-len(data))
            data = data + padding

        return np.array(data), np.array([self.labels[idx]])

In [7]:
train_dataset = ExeDataset(
    trainset["id"].tolist(), 
    train_data_path, 
    trainset["labels"].tolist(), 
    LEAVE_BIT_NUMBER
)
valid_dataset = ExeDataset(
    validset["id"].tolist(), 
    train_data_path, 
    validset["labels"].tolist(), 
    LEAVE_BIT_NUMBER
)
test_dataset = ExeDataset(
    testset["id"].tolist(), 
    test_data_path, 
    testset["labels"].tolist(), 
    LEAVE_BIT_NUMBER
)

In [8]:
trainloader = DataLoader(
    dataset = train_dataset,
    batch_size = BATCH_SIZE,
    shuffle = False,
    num_workers = NUM_WORKERS,
    pin_memory = True
)
validloader = DataLoader(
    dataset = valid_dataset,
    batch_size = BATCH_SIZE,
    shuffle = False,
    num_workers = NUM_WORKERS,
    pin_memory = True
)
testloader = DataLoader(
    dataset = test_dataset,
    batch_size = BATCH_SIZE,
    shuffle = False,
    num_workers = NUM_WORKERS,
    pin_memory = True
)

In [9]:
def mp_func(i,inpu,te,gr):
    cum_grad = 0
    for j,(inp,g,t) in enumerate(zip(inpu,gr,te)):
        if inp != 0: continue
        max_idx = np.argmin(g).tolist()
        org_max_idx = np.argmax(t).tolist()
        cum_grad += np.mean(g)
        if g[max_idx] > 0: continue
        te[j][org_max_idx] = 0
        te[j][max_idx] = 1
    return [i,te,cum_grad]

In [10]:
class Model(nn.Module):
    def __init__(self, data_length = 2e6, kernel_size = 500):
        super().__init__()
        self.embedding = nn.Embedding(257, 8, padding_idx=0)
        self.conv_layer_1 = nn.Conv1d(4, 128, kernel_size, stride = kernel_size, bias = True)
        # self.bn_1 = nn.BatchNorm1d(128)
        self.conv_layer_2 = nn.Conv1d(4, 128, kernel_size, stride = kernel_size, bias = True)
        self.pool_layer_2 = nn.MaxPool1d(data_length//kernel_size)
        self.fc_layer_3 = nn.Linear(128, 128)
        self.fc_layer_4 = nn.Linear(128, 2)

    def forward(self, input_, loss_fn):
        one_hot_x = F.one_hot(input_,num_classes=257).float()
        one_hot_x.requires_grad = True
        one_hot_x.retain_grad()
        batch_acc = []
        batch_grad = []
        for _ in range(13):
            x = one_hot_x @ self.embedding.weight
            x = torch.transpose(x, -1, -2)

            x_conv_1 = self.conv_layer_1(x[:,:4,:])
            x_conv_2 = torch.sigmoid(self.conv_layer_2(x[:,4:,:]))

            x = x_conv_1*x_conv_2
            del x_conv_1,x_conv_2
            x = self.pool_layer_2(x).squeeze()
            
            x = self.fc_layer_3(x)
            x = self.fc_layer_4(x)
            
            acc = torch.argmax(torch.softmax(x,dim=-1),dim=-1).float().mean()
            batch_acc.append(acc.tolist())
            
            mislead_labels = torch.cat((torch.ones(len(x)).reshape(-1,1),torch.zeros(len(x)).reshape(-1,1)),dim=1).cuda()
            loss = loss_fn(x,mislead_labels)
            loss.backward()
            
            padding_place_mask = (input_ == 0)
            shape = padding_place_mask.shape
            expand_padding_place_mask = padding_place_mask.unsqueeze(-1).expand(shape[0],shape[1],257)
            grad_min_idx = torch.argmin(one_hot_x.grad,dim=-1)
            grad_sez_mask = (one_hot_x.grad <= 0)
            grad_mask = (expand_padding_place_mask & grad_sez_mask)
            batch_grad.append(torch.div(
                input=(expand_padding_place_mask.float()*one_hot_x.grad).sum(),
                other=expand_padding_place_mask.float().sum()
            ).detach().cpu().numpy())
            new_x = torch.argmin(grad_mask.float()*one_hot_x.grad,dim=-1)
            grad_all_gz_mask = grad_sez_mask.sum(dim=-1) == 257
            mask = ((~grad_all_gz_mask) & padding_place_mask)
            temp = (1 - mask.float())*input_ + mask.float()*new_x
            one_hot_x.data = F.one_hot(temp.long(),num_classes=257).float()
            
            one_hot_x.grad.zero_()
            
        x = one_hot_x @ self.embedding.weight
        x = torch.transpose(x, -1, -2)

        x_conv_1 = self.conv_layer_1(x[:,:4,:])
        x_conv_2 = torch.sigmoid(self.conv_layer_2(x[:,4:,:]))

        x = x_conv_1*x_conv_2
        del x_conv_1,x_conv_2
        x = self.pool_layer_2(x).squeeze()

        x = self.fc_layer_3(x)
        x = self.fc_layer_4(x)
        return x.detach().cpu().numpy(),batch_acc,batch_grad

In [11]:
model = Model(data_length=LEAVE_BIT_NUMBER,kernel_size=KERNEL_SIZE)

ce_loss = nn.CrossEntropyLoss()
optim = Adam(model.parameters())

model = model.cuda() if CUDA else model
ce_loss = ce_loss.cuda() if CUDA else ce_less

In [12]:
model.load_state_dict(torch.load(trained_model_path + best_trained_model))

<All keys matched successfully>

In [13]:
model.eval()
acc = []
preds = []
labels = []
total_batch_acc = []
total_batch_grad = []
bar = tqdm(testloader)
# bar = tqdm(validloader)
for step, (batch_data,batch_label) in enumerate(bar):
    batch_data = batch_data.cuda() if CUDA else batch_data
    batch_label = batch_label.cuda() if CUDA else batch_label
    batch_label = batch_label.squeeze()
    temp_label = torch.zeros((len(batch_label),2))
    for idx,target in enumerate(batch_label.squeeze()): temp_label[idx,target] = 1
    temp_label = temp_label.cuda() if CUDA else temp_label

#     pred,input_,one_hot_x = model(batch_data,ce_loss)
#     break
    pred,batch_acc,batch_grad = model(batch_data,ce_loss)
    pred = np.argmax(pred,1)
    batch_label = batch_label.cpu().data.numpy()
    
    preds.extend(pred.tolist())
    labels.extend(batch_label.tolist())
    
    temp_acc = (batch_label == pred).mean()
    acc.append(temp_acc)
    total_batch_acc.append(batch_acc + [temp_acc])
    total_batch_grad.append([0] + batch_grad)
    total_batch_acc_str = '[' + ' '.join(map(lambda x: '%.10f'%x,np.mean(total_batch_acc,axis=0))) + ']'
    total_batch_grad_str = '[' + ' '.join(map(lambda x: '%.10f'%x,np.mean(total_batch_grad,axis=0))) + ']'
    print(total_batch_acc_str)
    print(total_batch_grad_str)
    print(f"test：{temp_acc:4f}, test mean: {np.mean(acc):4f}")

  0%|          | 1/1250 [00:05<1:53:47,  5.47s/it]

[0.7875000238 0.0250000004 0.0250000004 0.0250000004 0.0250000004 0.0250000004 0.0250000004 0.0250000004 0.0625000000 0.0250000004 0.0250000004 0.0250000004 0.0500000007 0.0250000000]
[0.0000000000 -0.0000000270 0.0000000001 0.0000000001 0.0000000001 0.0000000001 -0.0000000000 0.0000000000 -0.0000000000 -0.0000000006 0.0000000000 0.0000000001 0.0000000000 -0.0000000023]
test：0.025000, test mean: 0.025000


  0%|          | 2/1250 [00:07<1:14:10,  3.57s/it]

[0.8375000060 0.0250000004 0.0125000002 0.0125000002 0.0187500003 0.0125000002 0.0125000002 0.0125000002 0.0375000001 0.0125000002 0.0250000004 0.0125000002 0.0250000004 0.0125000000]
[0.0000000000 -0.0000000257 0.0000000001 0.0000000001 0.0000000001 0.0000000001 -0.0000000000 0.0000000000 -0.0000000000 -0.0000000008 -0.0000000000 -0.0000000004 -0.0000000000 -0.0000000011]
test：0.000000, test mean: 0.012500


  0%|          | 3/1250 [00:09<55:32,  2.67s/it]  

...

[0.8294270949 0.0204627408 0.0132311701 0.0123798079 0.0143229169 0.0122095355 0.0164963946 0.0120392630 0.0179186702 0.0120292470 0.0163762022 0.0119290868 0.0225460742 0.0119791667]
[0.0000000000 -0.0000000289 0.0000000003 0.0000000000 -0.0000000000 -0.0000000000 -0.0000000000 -0.0000000003 -0.0000000000 -0.0000000003 -0.0000000000 -0.0000000003 -0.0000000000 -0.0000000005]
test：0.012500, test mean: 0.011979


100%|█████████▉| 1249/1250 [30:58<00:01,  1.49s/it]

[0.8293334784 0.0204563655 0.0132305847 0.0123799042 0.0143214574 0.0122097680 0.0165032029 0.0120396319 0.0179143318 0.0120296239 0.0163730988 0.0119295439 0.0225380309 0.0119795837]
[0.0000000000 -0.0000000289 0.0000000003 0.0000000000 -0.0000000000 -0.0000000000 -0.0000000000 -0.0000000003 -0.0000000000 -0.0000000003 -0.0000000000 -0.0000000003 -0.0000000000 -0.0000000005]
test：0.012500, test mean: 0.011980


100%|██████████| 1250/1250 [31:00<00:00,  1.49s/it]

[0.8292800116 0.0204700004 0.0132300003 0.0123700002 0.0143100003 0.0122000002 0.0165000003 0.0120300002 0.0179000004 0.0120200002 0.0163600003 0.0119200002 0.0225400005 0.0119700000]
[0.0000000000 -0.0000000289 0.0000000003 0.0000000000 -0.0000000000 -0.0000000000 -0.0000000000 -0.0000000003 -0.0000000000 -0.0000000003 -0.0000000000 -0.0000000003 -0.0000000000 -0.0000000005]
test：0.000000, test mean: 0.011970


In [14]:
# import pickle

In [16]:
# with open('/workdir/security/home/junjiehuang2468/paper/results/ember/my_way/not_cumulative_gradients_v2_13iter_acc.txt','wb') as fp:
#     pickle.dump(total_batch_acc,fp)